<a href="https://colab.research.google.com/github/Oladimeji-Williams/HDSCFall2022/blob/main/MACHINE_LEARNING_REGRESSION_%7C_HamoyeFall22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage B Quiz Solution

Oladimeji Williams
© ellipsis

---

I **Oladimeji WILLIAMS**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [Code of Conduct](https://drive.google.com/file/d/1sbR80aowp1daCnElwx3kNm0fxids0e6b/view) contained therein.


### Overview: Machine Learning: Regression - Predicting Energy Efficiency of Buildings
> The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.

Attribute Information:
- 1.   `date`, time year-month-day hour:minute:second
- 2.   `Appliances`, energy use in Wh
- 3.   `lights`, energy use of light fixtures in the house in Wh
- 4.   `T1`, Temperature in kitchen area, in Celsius
- 5.   `RH_1`, Humidity in kitchen area, in %
- 6.   `T2`, Temperature in living room area, in Celsius
- 7.   `RH_2`, Humidity in living room area, in %
- 8.   `T3`, Temperature in laundry room area
- 9.   `RH_3`, Humidity in laundry room area, in %
- 10.  `T4`, Temperature in office room, in Celsius
- 11.  `RH_4`, Humidity in office room, in %
- 12.  `T5`, Temperature in bathroom, in Celsius
- 13.  `RH_5`, Humidity in bathroom, in %
- 14.  `T6`, Temperature outside the building (north side), in Celsius
- 15.  `RH_6`, Humidity outside the building (north side), in %
- 16.  `T7`, Temperature in ironing room , in Celsius
- 17.  `RH_7`, Humidity in ironing room, in %
- 18.  `T8`, Temperature in teenager room 2, in Celsius
- 19.  `RH_8`, Humidity in teenager room 2, in %
- 20.  `T9`, Temperature in parents room, in Celsius
- 21.  `RH_9`, Humidity in parents room, in %
- 22.  `T_out`, Temperature outside (from Chievres weather station), in Celsius
- 23.  `Press_mm_hg` (from Chievres weather station), in mm Hg
- 24.  `RH_out`, Humidity outside (from Chievres weather station), in %
- 25.  `Windspeed` (from Chievres weather station), in m/s
- 26.  `Visibility` (from Chievres weather station), in km
- 27.  `Tdewpoint` (from Chievres weather station), Â°C
- 28.  `rv1`, Random variable 1, nondimensional
- 29.  `rv2`, Random variable 2, nondimensional

# Preliminaries

In [1]:
# Load All Possible Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import io
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.correlation import plot_corr
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
uploaded = files.upload()

Saving energydata_complete.csv to energydata_complete.csv


In [4]:
# Load the dateset
df = pd.read_csv(io.BytesIO(uploaded["energydata_complete.csv"]), parse_dates=[0])

In [5]:
# Copy the dataset into another dataframe
df_copy = df.copy()

In [6]:
# Peak the first few observations of the dataset
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [7]:
# Peak the datatypes
df.dtypes

date           datetime64[ns]
Appliances              int64
lights                  int64
T1                    float64
RH_1                  float64
T2                    float64
RH_2                  float64
T3                    float64
RH_3                  float64
T4                    float64
RH_4                  float64
T5                    float64
RH_5                  float64
T6                    float64
RH_6                  float64
T7                    float64
RH_7                  float64
T8                    float64
RH_8                  float64
T9                    float64
RH_9                  float64
T_out                 float64
Press_mm_hg           float64
RH_out                float64
Windspeed             float64
Visibility            float64
Tdewpoint             float64
rv1                   float64
rv2                   float64
dtype: object

## Question 1

Coefficient of Determination

## Question 2

Y - Yhat

## Question 3

b = 0

## Question 4

Linear Regression with varying error terms

## Question 5

1 only

## Question 6

Lasso regression uses subset selection of features

## Question 7

Linear regression is sensitive to outliers

## Question 8

1, 2, 3 and 4

## Question 9

75% of the variance in Y is explained by X

## Question 10

Standard deviation of the residuals

## Question 11

Weights are regularized with the L1 norm

## Question 12

In [9]:
# Feature Selection for Linear Regression
X = df[["T2"]]
y = df["T6"]

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Build a Linear Regression Model
linear_model1 = LinearRegression()

# Fit the Linear Regression Model
linear_model1.fit(X_train, y_train)

# Use the model to make predictions on the test dataset
y_pred = linear_model1.predict(X_test)

# Print the r_squared value
print(f"r_squared is: {round(r2_score(y_test, y_pred), 2)}")

r_squared is: 0.64


## Question 13

In [10]:
# Drop the unwanted columns
df1 = df.drop(["date", "lights"], axis=1)

# Normalize the dataset with MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)

# Feature Selection
X = normalised_df.drop(["Appliances"], axis=1)
y = normalised_df["Appliances"]

# Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Model
linear_model2 = LinearRegression()
linear_model2.fit(X_train, y_train)

# Use the model to make predictions on the test dataset 
y_pred = linear_model2.predict(X_test)

# Print the mean_absolute_error
print(f"mean_absolute_error is: {round(mean_absolute_error(y_test, y_pred), 2)}")

mean_absolute_error is: 0.05


## Question 14

In [11]:
print(f"residual sum of squares is : {round(np.sum(np.square(y_test - y_pred)), 2)}")

residual sum of squares is : 45.35


## Question 15

In [12]:
print(f"Root mean square error: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)}")

Root mean square error: 0.088


## Question 16

In [13]:
print(f"coefficient of determination (r squared) is: {round(r2_score(y_test, y_pred), 2)}")

coefficient of determination (r squared) is: 0.15


## Question 17

In [14]:
## Compute a dataframe of weights
weights_df = pd.DataFrame(data={"weights":linear_model2.coef_}, index=X_train.columns)

## Print the lowest and highest weights
print(weights_df["weights"].idxmin(), ",", weights_df["weights"].idxmax())

RH_2 , RH_1


## Question 18

In [15]:
## Train a Ridge Regressor model
ridge_model = Ridge(alpha=0.4)

## Fit the model
ridge_model.fit(X_train, y_train)

## Use the model to make predictions on the test dataset
y_pred = ridge_model.predict(X_test)

## Print the root_mean_squared_error
print(f"Root mean square error: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)}")

Root mean square error: 0.088


## Question 19

In [16]:
## Train a Lass Regressor model
lasso_model = Lasso(alpha = 0.001)
lasso_model.fit(X_train, y_train)

## Compute a dataframe of weights
weights_df = pd.DataFrame(data={"weights":lasso_model.coef_}, index=X_train.columns)

np.sum(weights_df["weights"] != 0)

4

## Question 20

In [17]:
y_pred = lasso_model.predict(X_test)
print(f"Root mean square error: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)}")

Root mean square error: 0.094
